In [2]:
!ls

Topic_Modeling_Sentiment_Predict_Amazon_reviews.ipynb
neg_tweets.txt
pos_tweets.txt
reviews_Electronics_5.json


In [3]:
reset -sf

In [4]:
%pylab inline
import pandas as pd
import numpy as np
import json

Populating the interactive namespace from numpy and matplotlib


### Download Data

In [5]:
data = ''
with open('reviews_Electronics_5.json', 'r') as f:
    for line in f.readlines():
        data += line
        
df = pd.read_json(data, lines=True)
len(df)

1689188

In [6]:
df.head(3)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0528881469,"[0, 0]",5,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200
1,0528881469,"[12, 15]",1,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
2,0528881469,"[43, 45]",3,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400


In [7]:
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition.online_lda import LatentDirichletAllocation

### Preprocess for Data

In [8]:
def re_tokenize_and_clean(df_review):
    '''
    1. making lowercase, lower()
    2. tokenize each review into words(tokens) 
       w/ regexp(regular expression) in order to leave only words, clean [, . '' "" () digits]
    3. stemming, clean word format such as the past or countinous forms
    
    Param: df_review (pandas.series)
    
    '''
    RT = RegexpTokenizer(r'\b[^\d\W]+\b')
    PS = PorterStemmer()
    input_review = df_review.apply(lambda x: str(x).lower())  \
                              .apply(lambda x: RT.tokenize(x))  \
                              .apply(lambda x: [PS.stem(i) for i in x]) \
                              .apply(lambda x: str(x).replace("', '", " ") \
                                                     .replace("['", " ")   \
                                                     .replace("']", " "))
    return input_review

In [9]:
#df = df[:200]

In [10]:
df['input_review'] = re_tokenize_and_clean(df['reviewText'])

In [11]:
df['input_review'][:2]

0     we got thi gp for my husband who is an otr ov...
1     i m a profession otr truck driver and i bough...
Name: input_review, dtype: object

### Vectorizer & Fitting

In [12]:
'''
Two vecterizer used as vertorizer, Count(Tf) and Tf-idf, in LDA classification model.
param: max_features - number of top max_features for vocabulary 
       stop_word    - cleaning words
'''
# Count Verterizer
tf_vec = CountVectorizer(max_features=1000, stop_words='english')
tf_fit = tf_vec.fit_transform(df['input_review'].values)
print(tf_fit.shape)

# Tf-idf Vectorizer
tfidf_vec = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_fit = tfidf_vec.fit_transform(df['input_review'].values)
print(tfidf_fit.shape)

(1689188, 1000)
(1689188, 1000)


> LDA with Count Vectorizer performed usually better as compare with Tfidf Vectorizer LDA. <br>
I think because it is a probabilistic graphical model.<br>
It means that probabilistic graphical model (LDA) is not require idf value, give weight for words based on frequency in other documents.<br>
Therefore, Count (Tf) vectorizer is a good for LDA. 

### LDA w/ Tf

In [13]:
'''
param: n_components    - number of groups for categorize
       max_iter        - maxium number of iterations
       learning_method - update method for components; batch(0.20 folds) & online(faster)
       learning_offset - parameter that downweights early iterations in learning
       random_state    - the random number generator, like serier number for generator
'''
lda = LatentDirichletAllocation(n_components=5,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=77).fit(tf_fit)

In [14]:
def top_words(model, tf_words, n_top_words=13):
    for ix, component in enumerate(model.components_):
        print('============================='*4)
        print("Components {0}:".format(ix+1))
        print(" | ".join([tf_words[i] for i in component.argsort()[:-n_top_words - 1:-1]]))
        print('============================='*4, '\n')

print("\nComponents in LDA_Tf model: \n")
tf_words = tf_vec.get_feature_names()
top_words(lda, tf_words)


Components in LDA_Tf model: 

Components 1:
thi | wa | work | cabl | tv | connect | use | devic | great | just | set | router | product

Components 2:
camera | thi | use | veri | wa | pictur | len | light | good | qualiti | like | video | great

Components 3:
sound | thi | speaker | use | good | headphon | qualiti | veri | music | like | wa | great | bluetooth

Components 4:
thi | use | drive | usb | wa | card | work | comput | window | time | ha | instal | just

Components 5:
thi | case | use | ipad | tablet | batteri | like | charg | veri | keyboard | wa | fit | screen



In [15]:
# check the balance of performance
lda_vec = lda.fit_transform(tf_fit)
Counter([np.argmax(i) for i in lda_vec])

Counter({0: 453432, 1: 238197, 2: 246294, 3: 334256, 4: 417009})

> It seems like documents (reviews) distribution is blanced, but I will try another LDA and NMF with Tfidf to compare with this.

### Visualization - LDA w/ Count (Tf) Vectorize

In [16]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [17]:
# Visualize result of classification
pyLDAvis.sklearn.prepare(lda, tf_fit, tf_vec)

/Users/yunsup.jung/anaconda3/envs/hulu/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      23.971773        1       1 -0.102298 -0.041074
0      23.532433        1       2 -0.093702 -0.061090
4      18.063455        1       3  0.158429 -0.085400
1      17.654906        1       4  0.022244  0.105396
2      16.777433        1       5  0.015327  0.082167, topic_info=     Category           Freq       Term         Total  loglift  logprob
term                                                                   
116   Default  523144.000000     camera  5.231440e+05  30.0000  30.0000
126   Default  483700.000000       case  4.837000e+05  29.0000  29.0000
832   Default  453098.000000      sound  4.530980e+05  28.0000  28.0000
113   Default  371353.000000       cabl  3.713530e+05  27.0000  27.0000
249   Default  355220.000000      drive  3.552200e+05  26.0000  26.0000
836   Default  267878.000000    speaker  2.678780e+05  25.0000  25.0000
925   Default  327620.000000         tv  3.276200e+05  24.0000  24.0000
446   Default  232557.000000       ipad  2.325570e+05  23.0000  23.0000
883   Default  226588.000000     tablet  2.265880e+05  22.0000  22.0000
123   Default  225399.000000       card  2.253990e+05  21.0000  21.0000
136   Default  229311.000000      charg  2.293110e+05  20.0000  20.0000
458   Default  223268.000000   keyboard  2.232680e+05  19.0000  19.0000
393   Default  173434.000000   headphon  1.734340e+05  18.0000  18.0000
71    Default  264014.000000    batteri  2.640140e+05  17.0000  17.0000
172   Default  336460.000000    connect  3.364600e+05  16.0000  16.0000
194   Default  179903.000000      cover  1.799030e+05  15.0000  15.0000
563   Default  151896.000000      music  1.518960e+05  14.0000  14.0000
478   Default  144346.000000        len  1.443460e+05  13.0000  13.0000
977   Default  167805.000000     window  1.678050e+05  12.0000  12.0000
641   Default  197698.000000     pictur  1.976980e+05  11.0000  11.0000
326   Default  223105.000000        fit  2.231050e+05  10.0000  10.0000
937   Default  303461.000000        usb  3.034610e+05   9.0000   9.0000
686   Default  132727.000000    protect  1.327270e+05   8.0000   8.0000
748   Default  145451.000000     router  1.454510e+05   7.0000   7.0000
693   Default  386307.000000    qualiti  3.863070e+05   6.0000   6.0000
259   Default  110668.000000        ear  1.106680e+05   5.0000   5.0000
619   Default  106056.000000       pair  1.060560e+05   4.0000   4.0000
762   Default  352293.000000     screen  3.522930e+05   3.0000   3.0000
951   Default  102356.000000      volum  1.023560e+05   2.0000   2.0000
461   Default  105481.000000      kindl  1.054810e+05   1.0000   1.0000
...       ...            ...        ...           ...      ...      ...
335    Topic5   13271.862730         fm  1.327260e+04   1.7851  -6.8783
878    Topic5   25316.506938   surround  2.531897e+04   1.7850  -6.2325
448    Topic5   65088.412657       ipod  6.577624e+04   1.7746  -5.2882
868    Topic5   19544.087596        sub  2.031925e+04   1.7462  -6.4913
88     Topic5  112407.830415  bluetooth  1.393489e+05   1.5703  -4.7418
51     Topic5   90487.010666      audio  1.208744e+05   1.4956  -4.9588
122    Topic5   53856.504998        car  6.735438e+04   1.5615  -5.4777
581    Topic5   54328.477661       nois  6.891168e+04   1.5474  -5.4689
453    Topic5   30947.145346       jack  3.641985e+04   1.6223  -6.0317
693    Topic5  164543.268568    qualiti  3.863072e+05   0.9317  -4.3608
636    Topic5  106332.735406      phone  2.351997e+05   0.9913  -4.7974
156    Topic5   50644.339579    comfort  7.655099e+04   1.3720  -5.5391
366    Topic5  176354.881015       good  6.015410e+05   0.5581  -4.2915
648    Topic5   77598.185233       play  1.939892e+05   0.8689  -5.1124
180    Topic5   66999.638648    control  1.556044e+05   0.9425  -5.2593
946    Topic5  153804.148496       veri  7.508452e+05   0.1996  -4.4283
938    Topic5  208443.847440        use  1.388792e+

> The result looks like a good fitted LDA model for Amazon classification.<br>
However, I will try Tf-idf vectorizer for LDA model for just see the result as experiment. <br>
I didn't expected get a better result than LDA with Count(Tf) vertorizer.

### LDA w/ Tf-idf

In [18]:
lda_01 = LatentDirichletAllocation(n_components=5,
                                   max_iter=5,
                                   learning_method='online',
                                   learning_offset=50.,
                                   random_state=77).fit(tfidf_fit)

In [19]:
print("\nComponents in LDA_Tfidf model: \n")
tf_words_01 = tfidf_vec.get_feature_names()
top_words(lda_01, tf_words_01)


Components in LDA_Tfidf model: 

Components 1:
tv | watch | hdmi | roku | remot | movi | player | netflix | video | stream | samsung | hd | dvd

Components 2:
case | thi | ipad | cover | protect | screen | fit | kindl | tablet | protector | veri | like | use

Components 3:
thi | drive | use | wa | keyboard | work | card | usb | devic | comput | instal | veri | window

Components 4:
sound | thi | camera | batteri | use | speaker | wa | good | great | veri | headphon | qualiti | like

Components 5:
cabl | work | charg | thi | great | charger | product | price | good | wa | plug | phone | usb



In [20]:
# check the balance of performance
lda_vec_01 = lda_01.fit_transform(tfidf_fit)
Counter([np.argmax(i) for i in lda_vec_01])

Counter({0: 58252, 1: 329648, 2: 560401, 3: 530166, 4: 210721})

In [21]:
# Visualize result of classification
pyLDAvis.sklearn.prepare(lda_01, tfidf_fit, tfidf_vec)

/Users/yunsup.jung/anaconda3/envs/hulu/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      32.253204        1       1 -0.055755 -0.035104
3      31.180724        1       2 -0.052347 -0.068079
1      17.457049        1       3 -0.137154 -0.070547
4      11.093292        1       4 -0.037035  0.201171
0       8.015731        1       5  0.282291 -0.027441, topic_info=     Category          Freq      Term          Total  loglift  logprob
term                                                                  
925   Default  40578.000000        tv   40578.000000  30.0000  30.0000
113   Default  43651.000000      cabl   43651.000000  29.0000  29.0000
126   Default  46182.000000      case   46182.000000  28.0000  28.0000
136   Default  36558.000000     charg   36558.000000  27.0000  27.0000
984   Default  74016.000000      work   74016.000000  26.0000  26.0000
960   Default  15909.000000     watch   15909.000000  25.0000  25.0000
948   Default  23191.000000     video   23191.000000  24.0000  24.0000
137   Default  18721.000000   charger   18721.000000  23.0000  23.0000
728   Default  15928.000000     remot   15928.000000  22.0000  22.0000
194   Default  21621.000000     cover   21621.000000  21.0000  21.0000
390   Default  15298.000000      hdmi   15298.000000  20.0000  20.0000
832   Default  35454.000000     sound   35454.000000  19.0000  19.0000
446   Default  27249.000000      ipad   27249.000000  18.0000  18.0000
686   Default  19015.000000   protect   19015.000000  17.0000  17.0000
371   Default  63229.000000     great   63229.000000  16.0000  16.0000
561   Default  13380.000000      movi   13380.000000  15.0000  15.0000
745   Default  11254.000000      roku   11254.000000  14.0000  14.0000
249   Default  28775.000000     drive   28775.000000  13.0000  13.0000
754   Default  14459.000000   samsung   14459.000000  12.0000  12.0000
649   Default  12814.000000    player   12814.000000  11.0000  11.0000
461   Default  16563.000000     kindl   16563.000000  10.0000  10.0000
862   Default  12313.000000    stream   12313.000000   9.0000   9.0000
937   Default  27508.000000       usb   27508.000000   8.0000   8.0000
641   Default  20433.000000    pictur   20433.000000   7.0000   7.0000
388   Default  13162.000000        hd   13162.000000   6.0000   6.0000
458   Default  22739.000000  keyboard   22739.000000   5.0000   5.0000
572   Default  10239.000000   netflix   10239.000000   4.0000   4.0000
653   Default  22588.000000      plug   22588.000000   3.0000   3.0000
636   Default  28758.000000     phone   28758.000000   2.0000   2.0000
648   Default  18131.000000      play   18131.000000   1.0000   1.0000
...       ...           ...       ...            ...      ...      ...
827    Topic5   5418.677232       son    6603.057894   2.3261  -4.7476
862    Topic5   9816.051874    stream   12313.656880   2.2971  -4.1534
649    Topic5  10198.717892    player   12814.032639   2.2955  -4.1152
561    Topic5  10622.639917      movi   13380.297383   2.2930  -4.0744
390    Topic5  12056.563585      hdmi   15298.184686   2.2856  -3.9478
218    Topic5   4339.361085   describ    5661.586430   2.2578  -4.9697
205    Topic5   4712.479168  daughter    6236.209050   2.2436  -4.8872
360    Topic5   5435.170670      gift    7229.154228   2.2385  -4.7445
135    Topic5   6918.261870   channel    9416.045754   2.2155  -4.5033
178    Topic5   2822.522650   content    3846.454599   2.2142  -5.3998
997    Topic5   4182.249901    youtub    5727.642697   2.2093  -5.0066
434    Topic5   4960.223761     input    6975.505858   2.1828  -4.8360
728    Topic5  11048.048277     remot   15928.041658   2.1579  -4.0352
388    Topic5   9041.066155        hd   13162.019675   2.1482  -4.2357
182    Topic5   2467.707982   convert    3598.132670   2.1466  -5.5341
754    Topic5   9307.055169   samsung   14459.845749   2.0832  -4.2067
555    Topic5   8419.828224   monitor   14784.439357   1.9608  -4.3068
948    Topic5   9988.200862  

### Try NMF for Comparing Model as Test

In [22]:
nmf = NMF(init="nndsvd",
          n_components=5,
          max_iter=200,
          random_state=77).fit(tfidf_fit)

In [23]:
def top_words(model, tf_words, n_top_words=13):
    for ix, component in enumerate(model.components_):
        print('============================='*4)
        print("Components {0}:".format(ix+1))
        print(" | ".join([tf_words[i] for i in component.argsort()[:-n_top_words - 1:-1]]))
        print('============================='*4, '\n')

print("\nComponents in NMF model: \n")
tf_words = tfidf_vec.get_feature_names()
top_words(nmf, tf_words)


Components in NMF model: 

Components 1:
thi | work | wa | use | drive | product | just | time | great | devic | usb | comput | instal

Components 2:
camera | len | batteri | thi | pictur | canon | use | shoot | video | photo | shot | bag | light

Components 3:
sound | speaker | headphon | good | qualiti | ear | veri | bass | great | music | price | listen | volum

Components 4:
case | ipad | fit | protect | cover | thi | kindl | veri | nice | like | screen | look | tablet

Components 5:
cabl | hdmi | tv | work | qualiti | connect | thi | price | great | need | connector | good | buy



In [24]:
# check the balance of performance
nmf_vec = nmf.fit_transform(tfidf_fit)
Counter([np.argmax(i) for i in nmf_vec])

Counter({0: 786802, 1: 221655, 2: 276781, 3: 257591, 4: 146359})

>It seems like a good balanced NMF model for documents(reviews) distribution.

### Conclusion for Amazon Review Classification

>I applied two vectorizer, Counter(Tf) and Tf-idf, and two different model, LDA and NMF.<br>
* LDA - Count(Tf) vectorizer : Since LDA is probabilistic graphical model, it is a standard Topic modeling process.
  - The result is a good fitting model because documents(reviews) distribution is balenced.<br><br>
* LDA - Tfidf vectorizer: Just try to see the result (LDA is not require for idf (weighted) values for words.
  - The result looks not really balenced. <br><br>
* NMF - Tfidf vectorizer: NMF is require Tfidf values because this algorithm is based on 2 matrix multiplication, frequency and weight of words.

### Sentiment Analysis

In [25]:
df.columns

Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime', 'input_review'],
      dtype='object')

In [26]:
'''
In order to train sentiment model, use text convertor function with Regexp Tokenizer.
param: s - a tweet (str)
'''
def convertor(s):
    RT = RegexpTokenizer(r'\b[^\d\W]+\b')
    return({w: True for w in (RT.tokenize(s.lower()))})

In [27]:
'''
For sentiment model training, use tweets.txt of positive and negative sets.
Make positive and negative lists with convertor function. 
'''
pos = []
with open("./pos_tweets.txt") as fp:
    for tweet in fp: 
        pos.append([convertor(tweet), 'pos'])
        
neg = []
with open("./neg_tweets.txt") as fn:
    for tweet in fn: 
        neg.append([convertor(tweet), 'neg'])

In [28]:
'''
80% for training dateset, contain pos & neg tweets.
20% for test dataset for validate model quality.
'''
train = pos[int(len(pos)*.2):] + neg[int(len(pos)*.2):]
test = pos[:int(len(pos)*.2)] + neg[:int(len(pos)*.2)]

In [29]:
from nltk.classify import NaiveBayesClassifier
'''
Use Navie Bayes classifier.
show_most_informatice_feartures() shows list of the most effective words for pos/neg from train data. 
'''
classifier = NaiveBayesClassifier.train(train)
classifier.show_most_informative_features()

Most Informative Features
                 awesome = True              pos : neg    =     29.8 : 1.0
                headache = True              neg : pos    =     21.5 : 1.0
                   thank = True              pos : neg    =     14.5 : 1.0
                      no = True              neg : pos    =     14.0 : 1.0
                       n = True              pos : neg    =     12.8 : 1.0
                    love = True              pos : neg    =     12.3 : 1.0
                      hi = True              pos : neg    =     11.8 : 1.0
               beautiful = True              pos : neg    =     11.1 : 1.0
                   loved = True              pos : neg    =     11.1 : 1.0
                   funny = True              pos : neg    =      9.4 : 1.0


In [30]:
# Test with df['reviewText'][0]
print(classifier.classify(convertor(df['reviewText'][0])))

pos


In [31]:
'''
Validate sentiment model.
Shows accuracy score for test set.
This accuracy actually based on the trained tweet, so that might not really accurate for amazon review.
In order to improve the model accuracy for amazon review, it needs suitable kinds of date set.
'''
from nltk.classify.util import accuracy
print(accuracy(classifier, test))

0.8414634146341463


In [32]:
df.head(1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,input_review
0,0528881469,"[0, 0]",5,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",AO94DHGC771SJ,amazdnu,Gotta have GPS!,1370131200,we got thi gp for my husband who is an otr ov...


In [33]:
len(df), len(df['asin'].unique())

(1689188, 63001)

In [34]:
df_sen = df[['asin', 'reviewText', 'reviewTime']]
df_sen.head(3)

,asin,reviewText,reviewTime
0,0528881469,We got this GPS for my husband who is an (OTR)...,"06 2, 2013"
1,0528881469,"I'm a professional OTR truck driver, and I bou...","11 25, 2010"
2,0528881469,"Well, what can I say. I've had this unit in m...","09 9, 2010"


In [35]:
len(df_sen['asin'][0])

10

In [36]:
# Check the length of asin(product ID).
for i in range(len(df_sen['asin'])):
    assert len(df_sen['asin'][i]) == 10

In [37]:
from datetime import date
'''
To break year and week from timestamp.
'''
df_sen['yr'] = pd.to_datetime(df_sen['reviewTime']).apply(lambda x: x.isocalendar()[0])
df_sen['wk'] = pd.to_datetime(df_sen['reviewTime']).apply(lambda x: x.isocalendar()[1])

/Users/yunsup.jung/anaconda3/envs/hulu/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yunsup.jung/anaconda3/envs/hulu/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
df_sen.head(3)

,asin,reviewText,reviewTime,yr,wk
0,0528881469,We got this GPS for my husband who is an (OTR)...,"06 2, 2013",2013,22
1,0528881469,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",2010,47
2,0528881469,"Well, what can I say. I've had this unit in m...","09 9, 2010",2010,36


In [39]:
df_sen = (df_sen.sort_values(['asin', 'yr', 'wk'], ascending=[True, True, True])).reset_index()

In [40]:
def sentiment_nextWeek_review(asin, df):
    '''
    This is function for predict sentiment of next a week reviews.
    The function use the sentiment model, trained by positive and negative tweets, 84,15% accrate for test tweet.
    However, the df, which consist of Amazon reviews, might different style as tweets, 
    so the accracy is not really confident.
    
    param: asin - productID (str)
           df   - dataframe which is already sorted (pd.DataFrame) 
    '''
    pos, neg, cnt, review_rate = 0, 0, 0, 0                  # number of positive, negative and total reviews
    if asin in df['asin'].unique():
        for i, a in enumerate(df['asin']):
            if asin == a:
                cnt += 1
                '''
                Using tweet sentiment model
                * sen values will be pos or neg
                '''
                sen = classifier.classify(convertor(df['reviewText'][i]))
                if sen == 'pos':
                    pos += 1
                if sen == 'neg':
                    neg += 1
            else:
                if cnt > 0:
                    '''
                    yr_gap : last reviewed year - first reviewed year
                    total_gap_wk : (year = 52 weeks) -> convert into weeks
                    ave_num_review_per_week : number of reviews per a week
                    review_rate : probability for posible review (a week)
                                  ex) if the ave_num_review_per_week value is 3.5, 
                                      the number of expected reviews will be 3.5 next week for particular asin(product)
                    '''
                    yr_gap = df['yr'][i-1] - df['yr'][i-cnt]
                    total_gap_wk = (yr_gap-1)*52 + (52-df['wk'][i-cnt]) + df['wk'][i-1]
                    #print(total_gap_wk)
                    ave_num_review_per_week = cnt/total_gap_wk    # expected values
                    review_rate = ave_num_review_per_week/cnt     # In order to calculate expected number of pos & neg reviews
                    #print(cnt)
                    break
    else:
        print('The product, (asin #: {0}) cannot find in the list of review.'.format(asin))

    return asin, pos, neg, cnt, review_rate

In [41]:
asin, pos, neg, cnt, review_rate = sentiment_nextWeek_review('0972683275', df_sen)
print(' Predict for Review Qualities for {0} : Next a Week'.format(asin))
print('==============='*4)
print(' Positive Review, E(pos): {0:.4f}\n Negative Review, E(neg): {1:.4f}\n Total Review,  E(Total): {2:.4f}' \
     .format(pos*review_rate, neg*review_rate, cnt*review_rate))
print('==============='*4)

 Predict for Review Qualities for 0972683275 : Next a Week
 Positive Review, E(pos): 0.1958
 Negative Review, E(neg): 0.4639
 Total Review,  E(Total): 0.6596


* The product, (asin #: 0972683275) can get 0.6596 reviews, 0.4639 positive and 0.1958 negative reviews, next 1 week.

In [42]:
# Test for first 30 products
uniq_prod = df_sen['asin'].unique()
uniq_prod[:30]

array(['0528881469', '0594451647', '0594481813', '0972683275',
       '1400501466', '1400501520', '1400501776', '1400532620',
       '1400532655', '140053271X', '1400532736', '1400599997',
       '1400698987', '1400699169', '1615527613', '3744295508',
       '3930992868', '3936710058', '6301977173', '7214047977',
       '7507825604', '7799813393', '8862935293', '8862936826',
       '8918010656', '9043413585', '9573212900', '9573212919',
       '9575871979', '9625993428'], dtype=object)

In [43]:
for i in uniq_prod[:20]:
    asin, pos, neg, cnt, review_rate = sentiment_nextWeek_review(i, df_sen)
    print(' \nPredict for Review Qualities for {0} : Next a Week'.format(asin))
    print('==============='*4)
    print(' Positive Review, E(pos): {0:.4f}\n Negative Review, E(neg): {1:.4f}\n Total Review,  E(Total): {2:.4f}' \
         .format(pos*review_rate, neg*review_rate, cnt*review_rate))
    print('==============='*4)
    print('* The product, (asin #: {0}) can get {1:.4f} reviews, \n  {2:.4f} positive and {3:.4f} negative reviews, next 1 week.'\
         .format(i, cnt*review_rate, pos*review_rate, neg*review_rate), '\n')

 
Predict for Review Qualities for 0528881469 : Next a Week
 Positive Review, E(pos): 0.0070
 Negative Review, E(neg): 0.0282
 Total Review,  E(Total): 0.0352
* The product, (asin #: 0528881469) can get 0.0352 reviews, 
  0.0070 positive and 0.0282 negative reviews, next 1 week. 

 
Predict for Review Qualities for 0594451647 : Next a Week
 Positive Review, E(pos): 0.0370
 Negative Review, E(neg): 0.1481
 Total Review,  E(Total): 0.1852
* The product, (asin #: 0594451647) can get 0.1852 reviews, 
  0.0370 positive and 0.1481 negative reviews, next 1 week. 

 
Predict for Review Qualities for 0594481813 : Next a Week
 Positive Review, E(pos): 0.0200
 Negative Review, E(neg): 0.1400
 Total Review,  E(Total): 0.1600
* The product, (asin #: 0594481813) can get 0.1600 reviews, 
  0.0200 positive and 0.1400 negative reviews, next 1 week. 

 
Predict for Review Qualities for 0972683275 : Next a Week
 Positive Review, E(pos): 0.1958
 Negative Review, E(neg): 0.4639
 Total Review,  E(Total): 0.